## Data processing 

In [106]:
import pandas as pd
import numpy as np

# columns
c_year = 'year'
c_district = 'District'
c_amount = 'Amount in USD (nominal)'
c_country = 'Financier Country'
c_population = 'Population (2011)'

# values
v_china = 'China (People\'s Republic of)'
v_india = 'India'

# aid dataset
aid = pd.read_csv('data/aid_projects.csv')
aid[c_year] = aid['Commitment Year']
aid = aid.drop(['Title', 'Sector Name', 'Funding Agencies', 'Source URLs', 'Commitment Year'], axis=1)
aid_first = aid[aid[c_year] <= 2012]
aid_second = aid[aid[c_year] > 2012]

# win dataset
win_first = pd.read_csv('data/winning_candidates_2064.csv')
win_first[c_year] = 2008
win_second = pd.read_csv('data/winning_candidates_2070.csv')
win_second[c_year] = 2013
win = pd.concat([win_first, win_second])

In [123]:
def get_distr(df: pd.DataFrame, col):
    districts = df[[c_district, col]]
    return districts.groupby([c_district]).sum()

def get_stats(df: pd.DataFrame, col):
    distr = get_distr(df, col)

    mean = distr.mean().to_numpy()
    sd = distr.std().to_numpy()
    min = distr.min().to_numpy()
    max = distr.max().to_numpy()
    mean_first = get_distr(df[df[c_year] <= 2012], col).mean().to_numpy()
    mean_second = get_distr(df[df[c_year] > 2012], col).mean().to_numpy()
    return np.array([mean, sd, min, max, mean_first, mean_second]).flatten()




stats = pd.DataFrame([
    get_stats(aid, c_amount),
    get_stats(aid[aid[c_country] == v_china], c_amount),
    get_stats(aid[aid[c_country] == v_india], c_amount),
], columns=['Mean', 'SD', 'Min', 'Max', 'Mean First', 'Mean Second'])




#get_districts(aid_projects[aid_projects[country] == china]).sort_values(by=[amount], ascending=False)
#get_districts(aid_projects[aid_projects[country] == india]).sort_values(by=[amount], ascending=False)
#get_districts(aid_projects[aid_projects[district] == 'Kaski'].sort_values(by=[amount], ascending=False))
 #/ aid[c_population]
aid


,Financier Country,Recipient,Recipient Region,Flow Class,Amount in USD (nominal),District,Development Region,Province,Location,Distance,Ethnicity,Population (2011),GDP (Value Added) Rs. in million,HDI,Government Expenditure,year,aid per capita
0,China (People's Republic of),Nepal,Asia,ODA,13318641,Kathmandu,Central,Bagmati,Kathmandu,114,Braham-Hill,1632640,196723.1,0.632,3983659.85,2017,13318641
1,China (People's Republic of),Nepal,Asia,ODA,8139169,Kathmandu,Central,Bagmati,Ranipokhari,256,Braham-Hill,1632640,196723.1,0.632,3983659.85,2017,8139169
2,China (People's Republic of),Nepal,Asia,ODA,15740500,Lamjung,West,Gandaki,Birendranagar,645,Gurung,153626,8057.9,0.507,876383.97,2017,15740500
3,China (People's Republic of),Nepal,Asia,ODA,147984895,Sindhupalchok,Central,Bagmati,Rasuwagadhi,"2,8",Tamang,264274,12976.4,0.455,954115.86,2017,147984895
4,China (People's Republic of),Nepal,Asia,ODA,153497322,Kaski,West,Gandaki,Pokhara,254,Braham-Hill,454992,31244.7,0.576,1414683.74,2016,153497322
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,India,Nepal,Asia,ODA,353108,Rupandehi,West,Lumbini,Biratnagar,"6,3",Braham-Hill,800430,40138.8,0.498,1581064.43,2015,353108
638,India,Nepal,Asia,ODA,353108,Parsa,Central,Madhesh,Nepalgunj,"5,4",Musalman,533174,30004.6,0.464,1296319.53,2015,353108
639,India,Nepal,Asia,ODA,353108,Morang,East,Nr 1,Morang,75,Braham-Hill,880229,49026.1,0.513,1713304.47,2015,353108
640,India,Nepal,Asia,ODA,86700,Kathmandu,Central,Bagmati,Kathmandu,140,Braham-Hill,1632640,196723.1,0.632,3983659.85,2015,86700
